In [1]:
# Basic libraries
import pandas as pd
import numpy as np
import os
import sqlite3


In [2]:
import kagglehub
path = kagglehub.dataset_download("carrie1/ecommerce-data")

Using Colab cache for faster access to the 'ecommerce-data' dataset.


In [5]:
from google.colab import files
uploaded=files.upload()

Saving E-commerce.csv to E-commerce.csv


In [7]:
df = pd.read_csv("E-commerce.csv", encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [15]:
before_rows_total = df.shape[0]

rows_before_dropping_duplicates = df.shape[0]
# Drop duplicates
df = df.drop_duplicates()
duplicate_rows = rows_before_dropping_duplicates - df.shape[0]

# Handle missing values
df = df.dropna(subset=["CustomerID","StockCode"])  # critical columns
df.fillna(0, inplace=True)

after_rows = df.shape[0]
before_rows = before_rows_total

In [17]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
df.columns = df.columns.str.lower().str.strip()


In [19]:
df["invoicedate"] = pd.to_datetime(df["invoicedate"])
df["customerid"] = df["customerid"].astype(int)
df["quantity"] = df["quantity"].astype(int)
df["unitprice"] = df["unitprice"].astype(float)


In [21]:
df["revenue"] = df["quantity"] * df["unitprice"]
df["margin"] = df["revenue"] * 0.20   # assumed 20% margin
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,revenue,margin
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,3.060
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,4.068
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,4.400
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,4.068
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,4.068


In [22]:
#Segment Flags
df["high_value_customer"] = df["revenue"] > 1000
df["bulk_order"] = df["quantity"] >= 10


In [23]:
customers = df[["customerid", "country"]].drop_duplicates()


In [24]:
orders = df[["invoiceno", "invoicedate", "customerid", "revenue", "margin"]]


In [27]:
products = df[["stockcode", "description", "unitprice"]].drop_duplicates()


In [29]:
import os

folders = ["raw", "processed", "output"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)


In [33]:

customers.to_csv("output/customers.csv", index=False)
orders.to_csv("output/orders.csv", index=False)
products.to_csv("output/products.csv", index=False)


In [31]:
!mv *.csv raw/


In [34]:
after_rows = df.shape[0]


In [35]:
print("Rows before cleaning:", before_rows)
print("Rows after cleaning:", after_rows)
print("Removed rows:", before_rows - after_rows)


Rows before cleaning: 536641
Rows after cleaning: 401604
Removed rows: 135037


#I tracked duplicate records and records with missing CustomerID, validated row counts before and after transformation, and stored rejected rows for traceability.”